## 看下 0-255 模型在具有高斯分布的数据集中的精度

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader,  Subset, RandomSampler, BatchSampler

from model.inceptionv4 import inceptionv4
from model.mobilenetv2 import mobilenetv2
from model.resnet import resnet18
from model.shufflenetv2 import shufflenetv2
from model.vgg import vgg9_bn
from s3_dataset import PlantDataSet, PlantDataSetB
import matplotlib.pyplot as plt

In [3]:
def get_pre(net, device, data_loader):
    '''
    得到整个测试集预测的结果，以及标签
    '''
    label_all = []
    pre_all = []
    with torch.no_grad():
        net.eval()
        for data in data_loader:
            images, labels = data
            images = images.float().to(device)
            labels = labels.long().to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            label_all.extend(labels.data.cpu().numpy())
            pre_all.extend(predicted.data.cpu().numpy())
    return pre_all, label_all

In [3]:
from sklearn.metrics import accuracy_score # 精度
from sklearn.metrics import confusion_matrix # 混淆矩阵

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
Func = [vgg9_bn, resnet18, shufflenetv2, mobilenetv2, inceptionv4]
Save_path = [
    '../model_save/plant_disease2/vgg.pth',
    '../model_save/plant_disease2/resnet18.pth',
    '../model_save/plant_disease2/shufflenetv2.pth',
    '../model_save/plant_disease2/mobilenetv2.pth',
    '../model_save/plant_disease2/inceptionv4.pth'
]
data_test_a = DataLoader(PlantDataSet(flag='test'),
                              batch_size=8,
                          shuffle=False)
data_test_b = DataLoader(PlantDataSetB(flag='test'),
                              batch_size=8,
                          shuffle=False)
for Index in range(5):
    # 导入模型和权重
    net = Func[Index]()
    path_saved_model = Save_path[Index]
    net.load_state_dict(torch.load(path_saved_model))
    net.to(device)
    pre, label = get_pre(net, device, data_test_a)
    pre, label = np.array(pre), np.array(label)
    print('预测精度为：{:.9f}'.format(accuracy_score(label, pre)))

预测精度为：0.964716595
预测精度为：0.988276804
预测精度为：0.988276804
预测精度为：0.985545186
预测精度为：0.972000911


In [6]:
ans = confusion_matrix(label, pre, labels=list(range(38)))
from sklearn.metrics import classification_report
print(classification_report(label, pre, labels=list(range(38))))

In [4]:
from sklearn.metrics import accuracy_score # 精度
from sklearn.metrics import confusion_matrix # 混淆矩阵

device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
Func = [vgg9_bn, resnet18, shufflenetv2, mobilenetv2, inceptionv4]
Save_path = [
    '../model_save/plant_disease_domain/vgg.pth',
    '../model_save/plant_disease_domain/resnet18.pth',
    '../model_save/plant_disease_domain/shufflenetv2.pth'
]
data_test_a = DataLoader(PlantDataSet(flag='test'),
                              batch_size=8,
                          shuffle=False)
data_test_b = DataLoader(PlantDataSetB(flag='test'),
                              batch_size=8,
                          shuffle=False)
for Index in range(3):
    # 导入模型和权重
    net = Func[Index]()
    path_saved_model = Save_path[Index]
    net.load_state_dict(torch.load(path_saved_model))
    net.to(device)
    pre, label = get_pre(net, device, data_test_b)
    pre, label = np.array(pre), np.array(label)
    print('预测精度为：{:.9f}'.format(accuracy_score(label, pre)))

预测精度为：0.881839810
预测精度为：0.925455987
预测精度为：0.846153846
